**<font size='6' color='red'> ch03 연관분석</font>**
- pip install apyori

# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도 있는지 분석하는 방법
- 활용분야 : 상품진열, 사기보험적발, 신상품 카테고리 구성 ...

```
{조건 : X, ex오렌지주스} -> {결과 : Y, ex:와인}
연관분석과 관련 지표
1. 지지도(support) : 전체 데이터 중에 조건결과 항목들의 포함된 거래 비율
    (X, Y)의 항목수 / 전체 데이터 수 ex : 0.2
2. 신뢰도(confidence) : 조건이 발생했을 때, 결과가 동시에 일어날 확률
    (X, Y)의 항목수 / 조건항목수 ex : 0.5
3. 향상도(lift) : 우연히 발생한 규칙은 아닌지 확인
    조건결과 지지도 / (조건지지도)*(결과지지도)
    <1 : 음의 상관관계
    >1 : 양의 상관관계
                                지지도     신뢰도     향상도
{오렌지주스} -> {와인}          1/5(0.2)   1/2(0.5)  0.2/0.4*0.6 = 0.2/0.24 = 0.833
```

In [3]:
%ls data\cf_basket.csv

 D 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 2D13-14F8

 D:\ai\source\10_자연어처리\data 디렉터리

2024-12-30  오전 09:03               119 cf_basket.csv
               1개 파일                 119 바이트
               0개 디렉터리  383,372,886,016 바이트 남음


# 연관분석 구현

In [75]:
# 트랜젝션 데이터 가져오기
import csv
transaction = []
with open('data/cf_basket.csv', 'r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
#     print(list(csvdata))
    for row in csvdata:
        transaction.append(row)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [76]:
from apyori import apriori
rules = apriori(transaction, min_support=0.15, min_confidence=0.1)
rules = list(rules)
len(rules)

18

In [80]:
rules[15]
# items=frozenset({'와인', '콜라', '맥주'}), support=0.2 : 이 세트가 함께 나타날 확율
# items_base : 조건(lhs)
# items_add : 결과(rhs)

RelationRecord(items=frozenset({'맥주', '와인', '콜라'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주', '와인', '콜라'}), confidence=0.2, lift=1.0), OrderedStatistic(items_base=frozenset({'맥주'}), items_add=frozenset({'와인', '콜라'}), confidence=0.5, lift=1.25), OrderedStatistic(items_base=frozenset({'와인'}), items_add=frozenset({'맥주', '콜라'}), confidence=0.33333333333333337, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'맥주', '와인'}), confidence=0.25, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '와인'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '콜라'}), items_add=frozenset({'와인'}), confidence=0.5, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'와인', '콜라'}), items_add=frozenset({'맥주'}), confidence=0.5, lift=1.25)])

In [19]:
row = rules[15]
support = row[1]
ordered_st = row[2]

for item in ordered_st:
    lhs = ', '.join([x for x in item[0]])
    rhs = ', '.join([x for x in item[1]])
    confidence = item[2]
    lift = item[3]
    if lift > 1 :
        print('{} => {} \t {} \t {} \t {}'.format(lhs, rhs, support, confidence, lift))

맥주 => 와인, 콜라 	 0.2 	 0.5 	 1.25
콜라 => 맥주, 와인 	 0.2 	 0.25 	 1.25
맥주, 와인 => 콜라 	 0.2 	 1.0 	 1.25
와인, 콜라 => 맥주 	 0.2 	 0.5 	 1.25


In [39]:
print('조건 ->   결과\t\t지지도(support) 신뢰도(confidence)\t향상도(lift)')
for row in rules :
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1 :
            print(f'{lhs:3} -> {rhs:10} \t {support}(support) \t {round(confidence,2)}(confidence) \t {round(lift,2)}(lift)')

조건 ->   결과		지지도(support) 신뢰도(confidence)	향상도(lift)
맥주  -> 콜라         	 0.4(support) 	 1.0(confidence) 	 1.25(lift)
콜라  -> 맥주         	 0.4(support) 	 0.5(confidence) 	 1.25(lift)
소주  -> 콜라         	 0.6(support) 	 1.0(confidence) 	 1.25(lift)
콜라  -> 소주         	 0.6(support) 	 0.75(confidence) 	 1.25(lift)
콜라  -> 맥주, 소주     	 0.2(support) 	 0.25(confidence) 	 1.25(lift)
맥주, 소주 -> 콜라         	 0.2(support) 	 1.0(confidence) 	 1.25(lift)
맥주  -> 와인, 콜라     	 0.2(support) 	 0.5(confidence) 	 1.25(lift)
콜라  -> 맥주, 와인     	 0.2(support) 	 0.25(confidence) 	 1.25(lift)
맥주, 와인 -> 콜라         	 0.2(support) 	 1.0(confidence) 	 1.25(lift)
와인, 콜라 -> 맥주         	 0.2(support) 	 0.5(confidence) 	 1.25(lift)
소주  -> 오렌지주스, 콜라  	 0.2(support) 	 0.33(confidence) 	 1.67(lift)
콜라  -> 오렌지주스, 소주  	 0.2(support) 	 0.25(confidence) 	 1.25(lift)
오렌지주스, 소주 -> 콜라         	 0.2(support) 	 1.0(confidence) 	 1.25(lift)
오렌지주스, 콜라 -> 소주         	 0.2(support) 	 1.0(confidence) 	 1.67(lift)
콜라  -> 소주, 와인     	 0.2(sup

In [43]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','support','confidence','lift'])
index=0

for row in rules :
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1 :
            rules_df.loc[index] = [lhs, rhs, support, confidence, lift]
            index += 1
rules_df.sort_values(by=['confidence', 'lift'], ascending=False)

,lhs,rhs,support,confidence,lift
13,"오렌지주스, 콜라",소주,0.2,1.000000,1.666667
0,맥주,콜라,0.4,1.000000,1.250000
2,소주,콜라,0.6,1.000000,1.250000
5,"맥주, 소주",콜라,0.2,1.000000,1.250000
8,"맥주, 와인",콜라,0.2,1.000000,1.250000
12,"오렌지주스, 소주",콜라,0.2,1.000000,1.250000
15,"소주, 와인",콜라,0.2,1.000000,1.250000
3,콜라,소주,0.6,0.750000,1.250000
1,콜라,맥주,0.4,0.500000,1.250000
6,맥주,"와인, 콜라",0.2,0.500000,1.250000


# 3. 뉴스 연관 분석
- http://fs.jtbc.joins.com/RSS/economy.xml 기사 검색 후 -> 명사 추출(기사, 앵커 제외) -> 연관분석 (min_support=0.15, min_confidence=0.1)

In [81]:
import requests
from bs4 import BeautifulSoup
rss_url = 'http://fs.jtbc.joins.com/RSS/economy.xml'

jtbc_economy = requests.get(rss_url)
soup = BeautifulSoup(jtbc_economy.content, 'xml')
link_ele = soup.select('title')
link_list = [link.text for link in link_ele]
link_list

['JTBC News',
 "둔촌주공 '전세 홍수' 예상했는데…눈치게임 시작됐다",
 '"실제 가보니 완전 사기네"…캠핑장 관리 소홀했다간',
 '"요즘 한 끼도 버겁다"…편의점 갔다가 \'이 정도일 줄은\'',
 '"오늘은 굶어야겠다" 말 나오는 이유…편의점 도시락도 \'7천원 시대\'',
 "경영권 분쟁 '주총 표대결'로…롤러코스터 탄 고려아연 주가",
 '"결혼하기 진짜 힘들겠구나"…\'돈 걱정\' 앞서는 예비부부들',
 "예비부부 울리는 '웨딩 물가'…올해 예식장 비용 21% 올라",
 "20대 '비정규직 비중' 역대 최고…10명 중 4명 꼴",
 '대통령까지 나서 "확실하게 살아난다"더니…경제 \'빨간불\'',
 '믿었던 수출마저 뒷걸음질…3분기 경제성장률 0.1% 그쳐',
 "'HBM' 날개 단 SK하이닉스…삼성전자 뛰어넘는 역대급 실적",
 '여전히 비싼 배추…정부 "11월 중순엔 3천원대로 내려갈 것"',
 "내년 추석 1년 남았는데…황금연휴 항공권 '예약 전쟁'",
 "역대급 여름 폭염 탓에…제철에 '귀한 몸' 된 가을 전어",
 "바다 뜨거워지자 귀해진 전어…현실화한 '피시플레이션'",
 "정부, '자본잠식' 티메프 재무 상태도 안 보고…80억 예산 책정",
 '"정부 믿고 팔았다 피해"…정부, 재무 상태 안 보고 티몬 지원',
 '15억 이상 아파트 거래, 최근 5년 중 최고…10건 중 8건은 서울',
 '서민 동아줄인데…오락가락 \'디딤돌 대출\'에 "불안"',
 '뉴욕증시 다우·S&P500 사상 최고치…6주 연속 상승세']

In [56]:
# link_list들의 뉴스 기사 중 명사만 뽑아 list로 생성
from konlpy.tag import Komoran, Kkma
from mecab import MeCab
komoran = MeCab()
news = []
stopwords = ['기자', '앵커']
for link in link_list:
    response = requests.get(link)
    news_soup = BeautifulSoup(response.content, 'html.parser')
    title = news_soup.select_one('title').text.replace(' | JTBC 뉴스', '')
    content = str(news_soup.select_one('meta[name="description"]'))
    content = content.replace('<meta content="', '').replace('" name="description"/>', '')
    noun_list = komoran.nouns(title + ' ' + content)
    noun_list = [word for word, tag in komoran.pos(title + ' ' + content) if tag in ('NNG', 'NNP')]
    noun_list = [noun for noun in noun_list if noun not in ('기자', '앵커')]
    news.append(noun_list)
print(news)

[['둔촌', '주공', '전세', '홍수', '예상', '눈치', '게임', '시작', '단군', '최대', '규모', '재건축', '둔촌', '주공', '아파트', '입주', '다음', '달', '시작', '규모', '입주', '전세', '매물', '현장', '분위기', '이상', '다음', '달', '세대', '입주', '시작', '서울', '둔촌동', '올림픽', '파크', '포레', '온', '국내', '건축', '가운데', '역대', '최대', '규모', '이용택', '공인', '중개사', '공개', '행사', '이후', '가량', '판단', '연령', '세대', '관심', '단지', '입주', '시기', '전', '전세', '물량', '지도', '관심', '전', '세대', '입주', '서울', '헬리오', '시티', '당시', '전세', '매매', '가격', '동반', '하락', '이번', '현장', '김복수', '공인', '중개사', '일반', '아파트', '입주', '정도', '전세', '보통', '시장', '모습', '전세', '입주', '전세', '본인', '가격', '거주', '의무', '폐지', '신축', '선호', '현상', '가세', '전세', '매물', '자체', '부족', '인터넷', '매물', '세대', '최근', '정부', '대출', '관리', '전세', '대출', '규제', '점', '한몫', '권대', '서강대', '부동산', '학과', '교수', '전세', '대출', '세입자', '물량', '전세', '대출', '실입', '주', '현상', '임대', '물량', '상대', '전세', '집', '주인', '주변', '전셋값', '신축', '중심', '추세', '함영진', '직방', '빅', '데이터', '랩', '장', '규모', '입', '주장', '전세', '가격', '하락', '매물', '유통', '기대', '영상', '취재', '승규', '영상', '편집', '김지우'], ['실제', '캠', '핑장', '관리',

In [63]:
transaction=[]
for row in news:
    transaction.append(row)
transaction

[['둔촌',
  '주공',
  '전세',
  '홍수',
  '예상',
  '눈치',
  '게임',
  '시작',
  '단군',
  '최대',
  '규모',
  '재건축',
  '둔촌',
  '주공',
  '아파트',
  '입주',
  '다음',
  '달',
  '시작',
  '규모',
  '입주',
  '전세',
  '매물',
  '현장',
  '분위기',
  '이상',
  '다음',
  '달',
  '세대',
  '입주',
  '시작',
  '서울',
  '둔촌동',
  '올림픽',
  '파크',
  '포레',
  '온',
  '국내',
  '건축',
  '가운데',
  '역대',
  '최대',
  '규모',
  '이용택',
  '공인',
  '중개사',
  '공개',
  '행사',
  '이후',
  '가량',
  '판단',
  '연령',
  '세대',
  '관심',
  '단지',
  '입주',
  '시기',
  '전',
  '전세',
  '물량',
  '지도',
  '관심',
  '전',
  '세대',
  '입주',
  '서울',
  '헬리오',
  '시티',
  '당시',
  '전세',
  '매매',
  '가격',
  '동반',
  '하락',
  '이번',
  '현장',
  '김복수',
  '공인',
  '중개사',
  '일반',
  '아파트',
  '입주',
  '정도',
  '전세',
  '보통',
  '시장',
  '모습',
  '전세',
  '입주',
  '전세',
  '본인',
  '가격',
  '거주',
  '의무',
  '폐지',
  '신축',
  '선호',
  '현상',
  '가세',
  '전세',
  '매물',
  '자체',
  '부족',
  '인터넷',
  '매물',
  '세대',
  '최근',
  '정부',
  '대출',
  '관리',
  '전세',
  '대출',
  '규제',
  '점',
  '한몫',
  '권대',
  '서강대',
  '부동산',
  '학과',
  '교수',
  '전세',
  '대출',
  '세입자',
  '물량'

In [64]:
from apyori import apriori
rules = apriori(transaction, min_support=0.15, min_confidence=0.1)
rules = list(rules)
len(rules)

27

In [73]:
rules

[RelationRecord(items=frozenset({'가격'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'가격'}), confidence=0.2, lift=1.0)]),
 RelationRecord(items=frozenset({'내년'}), support=0.15, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'내년'}), confidence=0.15, lift=1.0)]),
 RelationRecord(items=frozenset({'말'}), support=0.25, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'말'}), confidence=0.25, lift=1.0)]),
 RelationRecord(items=frozenset({'배'}), support=0.15, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'배'}), confidence=0.15, lift=1.0)]),
 RelationRecord(items=frozenset({'분기'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'분기'}), confidence=0.2, lift=1.0)]),
 RelationRecord(items=frozenset({'사실'}), support=0.15, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozens

In [70]:
print('조건 ->   결과\t\t지지도(support) 신뢰도(confidence)\t향상도(lift)')
for row in rules :
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1 :
            print(f'{lhs:3} -> {rhs:10} \t {support}(support) \t {round(confidence,2)}(confidence) \t {round(lift,2)}(lift)')

조건 ->   결과		지지도(support) 신뢰도(confidence)	향상도(lift)
가격  -> 이상         	 0.15(support) 	 0.75(confidence) 	 3.0(lift)
이상  -> 가격         	 0.15(support) 	 0.6(confidence) 	 3.0(lift)
가격  -> 최근         	 0.15(support) 	 0.75(confidence) 	 3.75(lift)
최근  -> 가격         	 0.15(support) 	 0.75(confidence) 	 3.75(lift)
말   -> 아람         	 0.15(support) 	 0.6(confidence) 	 2.4(lift)
아람  -> 말          	 0.15(support) 	 0.6(confidence) 	 2.4(lift)
사실  -> 상태         	 0.15(support) 	 1.0(confidence) 	 6.67(lift)
상태  -> 사실         	 0.15(support) 	 1.0(confidence) 	 6.67(lift)
사실  -> 업체         	 0.15(support) 	 1.0(confidence) 	 6.67(lift)
업체  -> 사실         	 0.15(support) 	 1.0(confidence) 	 6.67(lift)
상태  -> 업체         	 0.15(support) 	 1.0(confidence) 	 6.67(lift)
업체  -> 상태         	 0.15(support) 	 1.0(confidence) 	 6.67(lift)
사실  -> 업체, 상태     	 0.15(support) 	 1.0(confidence) 	 6.67(lift)
상태  -> 업체, 사실     	 0.15(support) 	 1.0(confidence) 	 6.67(lift)
업체  -> 상태, 사실     	 0.15(support) 	 1.0(

In [71]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','support','confidence','lift'])
index=0

for row in rules :
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift > 1 :
            rules_df.loc[index] = [lhs, rhs, support, confidence, lift]
            index += 1
rules_df.sort_values(by=['confidence', 'lift'], ascending=False)

,lhs,rhs,support,confidence,lift
6,사실,상태,0.15,1.00,6.666667
7,상태,사실,0.15,1.00,6.666667
8,사실,업체,0.15,1.00,6.666667
9,업체,사실,0.15,1.00,6.666667
10,상태,업체,0.15,1.00,6.666667
11,업체,상태,0.15,1.00,6.666667
12,사실,"업체, 상태",0.15,1.00,6.666667
13,상태,"업체, 사실",0.15,1.00,6.666667
14,업체,"상태, 사실",0.15,1.00,6.666667
15,"상태, 사실",업체,0.15,1.00,6.666667
